In [1]:
import Parser
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances
from nltk.tokenize import TreebankWordTokenizer

from functools import partial
import math
from tqdm import tqdm
import turbodbc

In [2]:
def create_connection_string_turbo(server, database):
    options = turbodbc.make_options(prefer_unicode=True)
    constr = 'Driver={ODBC Driver 13 for SQL Server};Server=' + \
        server + ';Database=' + database + ';Trusted_Connection=yes;'
    con = turbodbc.connect(connection_string=constr, turbodbc_options=options)
    return con

In [3]:
def batch(n, iterable):
    from scipy import sparse
    if sparse.issparse(iterable) or isinstance(
            iterable,
            (np.ndarray, np.generic)):
        row_l = iterable.shape[0]
        for ndx in range(0, row_l, n):
            yield iterable[ndx:min(ndx + n, row_l), ]

In [4]:
tree = Parser.get_tree("DebrunnerAciferBW-1515-2018-03-de.xml")
parser = Parser.XML_Parser(tree)
parser.parse_xml()
parser.dict_to_df("id")
parser.DF.to_clipboard(index=False)

In [5]:
df = parser.DF[['ArtikelId','Art_Txt_Kurz', 'Art_Txt_Lang','Preis']].copy()

In [6]:
query = """
set nocount on;

SELECT  [Artikelnummer]
      ,[Artikel Beschreibung]
      ,[Lieferantenname]
      ,[Bruttoverkauf]
      ,[Standardkosten]
      ,[Lieferantennummer]
      ,EAN = Barcode
into #temp
FROM [AnalystCM].[dbo].[LOOKUP_ArtikelstammCRHT]
where [Einmalartikel (J/N)] = 0 and [Artikel Status] = '02'
and Artikelgruppe not between 5000 and 5599

select idItemOrigin
,AvgPrice = Avg(Sales / case when Quantity = 0 then NULL else Quantity end)
,WAvgPrice = sum(Sales) / case when sum(Quantity) = 0 then NULL else sum(Quantity) end
,SalesLTM = sum(Sales)
,MarginLTM = sum(Margin)
into #avgprice
from crhbusadwh01.infopool.fact.sales
where date > dateadd(month,-12, getdate())
group by idItemOrigin

select [Artikelnummer]
      ,[Artikel Beschreibung]
      ,[Lieferantenname]
      ,[Bruttoverkauf]
      ,[Standardkosten]
      ,[ArtikelnummerLieferant] = [Lieferantennummer]
      ,EAN
      ,Durchschnittspreis = AvgPrice
      ,GDurchschnittspreis = WAvgPrice
      ,SalesLTM
      ,MarginLTM
from #temp t
    left join #avgprice a on t.Artikelnummer = a.idItemOrigin collate Latin1_General_CI_AS

drop table #temp
drop table #avgprice
"""

In [7]:
con = create_connection_string_turbo("CRHBUSADWH51", "Operations")
df_server = pd.read_sql(query, con)
df_server['Bruttoverkauf'] = df_server['Bruttoverkauf'].astype("float")
df_server['Standardkosten'] = df_server['Standardkosten'].astype("float")
df_server['EAN'] = df_server['EAN'].astype("str")

In [8]:
df['Art_Txt'] = df['Art_Txt_Lang'] + " " +df['Art_Txt_Kurz']

In [9]:
vec = TfidfVectorizer(tokenizer=TreebankWordTokenizer().tokenize)
vec.fit(pd.concat([df['Art_Txt'], df_server['Artikel Beschreibung']]))

X = vec.transform(df['Art_Txt_Lang'])
Y = vec.transform(df_server['Artikel Beschreibung'])

print(X.shape)
print(Y.shape)

(51621, 157822)
(72931, 157822)


In [10]:
# too memory heavy, no chance to do it without batching (32 GB Ram VM)
chunksize = 1000

batcher = partial(batch, chunksize)
idx = np.zeros(shape=(X.shape[0], 1), dtype=np.int)
dist = np.zeros(shape=(X.shape[0], 1), dtype=np.float)

print("Number of Batches = {}".format(math.ceil(X.shape[0] / chunksize)))

for i,j,v in tqdm(zip(batcher(idx),batcher(dist), batcher(X))):
    d = pairwise_distances(v, Y, metric='cosine')
    i[:,0] = d.argmin(axis=1)
    j[:,0] = d.min(axis=1)

Number of Batches = 52


52it [01:26,  1.67s/it]


In [11]:
df['idx_y'] = idx
df['dist_y'] = dist
mmap = dict(enumerate(df_server['Artikelnummer']))
df['Artikelnummer_CRHT'] = df['idx_y'].map(mmap)

In [12]:
df.head()

,,,ArtikelId,Art_Txt_Kurz,Art_Txt_Lang,Preis,Art_Txt,idx_y,dist_y,Artikelnummer_CRHT
10027422,,,10027422,Schlauch-Raccords mit Innengewinde und loser M...,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",6.30,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",44285,0.774604,3812615
10027424,,,10027424,Schlauch-Raccords mit Innengewinde und loser M...,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",6.70,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",68905,0.753880,2038822
10027425,,,10027425,Schlauch-Raccords mit Innengewinde und loser M...,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",6.90,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",68905,0.754721,2038822
10027426,,,10027426,Schlauch-Raccords mit Innengewinde und loser M...,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",6.95,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",68905,0.753339,2038822
10027427,,,10027427,Schlauch-Raccords mit Innengewinde und loser M...,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",7.75,"GEKA Schlauch-Raccords, mit Innengewinde, mit ...",57893,0.791357,6193350


In [13]:
df = df.merge(df_server, how="left",
              left_on="Artikelnummer_CRHT",
              right_on="Artikelnummer")

In [14]:
df = df.sort_values('dist_y', ascending=True)

In [15]:
df.to_excel("Debrunner_Matching.xlsx")